In [ ]:
from importlib import reload

import numpy as np
from plotly import express as px
from plotly import graph_objects as go
from plotly import io as pio
from sklearn import linear_model, metrics, datasets, preprocessing, model_selection

import cool_search as cool

reload(cool)


plot_temp = pio.templates["plotly_dark"]
plot_temp.layout.width = 700
plot_temp.layout.height = 500
plot_temp.layout.autosize = False
pio.templates.default = plot_temp

## linear regression

In [ ]:
rng = np.random.default_rng(seed=137)

X_train = np.linspace(0, 20, 80).reshape(-1, 1)
X_val = np.linspace(3, 22, 30).reshape(-1, 1)


def fun(x, sigma):
    return 3 + 0.8 * x + sigma * rng.normal(size=x.shape)


Y_true = fun(X_train, 0)
Y_train = fun(X_train, 2)
Y_val = fun(X_val, 2)


linreg = linear_model.ElasticNet(alpha=3.3, l1_ratio=1)
linreg.fit(X_train, Y_train)
y_linreg_default = linreg.predict(X_train.reshape(-1, 1)).flatten()


fig = go.Figure()

fig.add_traces(
    [
        go.Scatter(x=X_train.ravel(), y=Y_train.ravel(), mode="markers", name="data"),
        go.Scatter(x=X_train.ravel(), y=Y_true.ravel(), mode="lines", name="GT"),
        go.Scatter(
            x=X_train.ravel(), y=y_linreg_default.ravel(), mode="lines", name="linreg"
        ),
        go.Scatter(x=X_val.ravel(), y=Y_val.ravel(), mode="markers", name="validation"),
    ]
)

In [ ]:
reload(cool)
search = cool.search_one_dim(
    estimator=linear_model.ElasticNet,
    param_name="alpha",
    param_range=(0.0001, 10),
    metric="MSE",
    fixed_params={
        "l1_ratio": 1,
    },
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
)

pp, score, runtime = search.sample_grid(100)
print(search)
px.line(x=pp, y=score)
# px.line(x=pp, y=runtime)

In [ ]:
print(search.sample_point(12))
print(search)

## classification

In [ ]:
X, Y = datasets.make_classification(
    n_samples=1000,
    n_features=8,
    n_informative=4,
    n_classes=3,
    random_state=42,
)

X_train, X_val, Y_train, Y_val = model_selection.train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
)


px.scatter(
    x=X_train[:, 0],
    y=X_train[:, 1],
    color=[str(c) for c in Y_train],
    width=300,
    height=300,
)

In [ ]:
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)

logreg = linear_model.LogisticRegression(C=0.3)
logreg.fit(X_train, Y_train)
print(metrics.classification_report(Y_val, logreg.predict(X_val), digits=5))
print(cool.search_one_dim.accuracy_score(Y_val, logreg.predict(X_val)))

In [ ]:
reload(cool)
search = cool.search_one_dim(
    estimator=linear_model.LogisticRegression,
    param_name="C",
    param_range=(0.01, 2),
    metric="accuracy",
    fixed_params={},
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
)

In [ ]:
print(search)

pp, score, runtime = search.sample_grid(100)
px.scatter(x=pp, y=score).show()
px.scatter(x=pp, y=runtime).show()

In [ ]:
reload(cool)
search.sample_point(0.87434343)
print(search)

kde = search.sample_points_KDE(kernel="gaussian", bandwidth=0.1, resolution=1000)

d = (np.gradient(kde)) ** 2
d2 = np.gradient(np.gradient(kde))
movmean = np.convolve(kde, np.ones(7) / 7)[3:]

xx = np.linspace(*search.param_range, len(kde))

min = np.argsort(d)[:5]
print(min)
print(d2[min])
cand = np.where(d2[min] > 0)[0]
print(cand)
print(f"local minimum at: {xx[min[cand]]}")

fig = px.line(x=xx, y=kde)
fig.add_trace(
    go.Scatter(x=xx[min[cand]], y=kde[min[cand]], mode="markers", name="stat points")
)
print(kde.mean())
fig.add_trace(go.Scatter(x=xx, y=d))


## Gaussian process estimate

In [ ]:
rng = np.random.default_rng(seed=137)

X_train = np.linspace(0, 20, 80).reshape(-1, 1)
X_val = np.linspace(3, 22, 30).reshape(-1, 1)


def fun(x, sigma):
    return 3 + 0.8 * x + sigma * rng.normal(size=x.shape)


Y_true = fun(X_train, 0)
Y_train = fun(X_train, 2)
Y_val = fun(X_val, 2)


reload(cool)
search = cool.search_one_dim(
    estimator=linear_model.ElasticNet,
    param_name="alpha",
    param_range=(0.01, 10),
    metric="MSE",
    fixed_params={},
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
)
samp_grid_low = search.sample_grid(resolution=5)

res = 100
pred_mean, pred_std = search.estimate_score_GP(resolution=res)
search = cool.search_one_dim(
    estimator=linear_model.ElasticNet,
    param_name="alpha",
    param_range=(0.01, 10),
    metric="MSE",
    fixed_params={},
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
)
samp_grid_high = search.sample_grid(resolution=res)

go.Figure().add_traces(
    [
        go.Scatter(
            x=samp_grid_low["alpha"],
            y=samp_grid_low["MSE"],
            name="low res",
            mode="markers",
        ),
        go.Scatter(
            x=np.linspace(*search.param_range, res),
            y=pred_mean,
            name="GP",
        ),
        go.Scatter(
            x=samp_grid_high["alpha"],
            y=samp_grid_high["MSE"],
            name="high res",
            line=dict(dash="dash"),
        ),
    ]
).show()


In [ ]:
go.Figure().add_traces(
    [
        go.Scatter(
            x=np.linspace(*search.param_range, res),
            y=pred_mean - pred_std,
            name="lower",
            line=dict(color="green"),
        ),
        go.Scatter(
            x=np.linspace(*search.param_range, res),
            y=pred_mean + pred_std,
            name="upper",
            fill="tonexty",
            line=dict(color="green"),
        ),
        go.Scatter(
            x=samp_grid_low["alpha"],
            y=samp_grid_low["MSE"],
            name="low res",
            mode="markers",
            marker=dict(size=8),
        ),
        go.Scatter(
            x=np.linspace(*search.param_range, res),
            y=pred_mean,
            name="GP",
        ),
        go.Scatter(
            x=samp_grid_high["alpha"],
            y=samp_grid_high["MSE"],
            name="high res",
            line=dict(dash="dash"),
        ),
    ]
)
